# Introduction

## Time evolution

#### Describing how any quantum system evolves in time is determined by an initial state $\ket{\psi_0}$ and a Hamiltonian, $2^n \times 2^n$ Hermitean matrix. The state at a later time is calculated using the time-dependent Schrödinger equation:
#### $i \hbar \frac{\partial\ket{\psi(t)}}{\partial t} = H $$\ket{\psi(t)}$
#### This can be solved on a general form such that the state after a time t is given by
#### $\ket{\psi(t)} = e^{-iHt} \ket{\psi(0)}$
#### Here $\ket{\psi(0)}$ is the initial state you start from. This is a powerful equation that can simulate time evolution of quantum systems. However, the process quite quickly breakdown as the size of the Hamiltonian and by extension the states become to large to be simulated on computers. Quantum computers have some algorithms that can help with this scaling problem, but the Hamiltonian might be extremely complicated to implement as an unitary operation. In the following we will go through Trotter decomposition that approximates the Hamiltonians used in time evolution.

## Trotter decomposition

#### The Trotter decomposition is a method to decompose an expotential function of the sum of square matrices into a product of expotential function of square matrices. The first order Trotter formula is given by:
#### $e^{A+B} = \lim_{n\rightarrow} (e^\frac{A}{n}e^\frac{B}{n})^n$
#### This method can be done to different degrees of precision, the first order Trotter approximation of two square matrices A and B is given by:
#### $e^{\delta(A+B)} = e^{\delta A} e^{\delta B} + \mathcal{O}(\delta^2)$
#### The error term is needed since the matrices A and B do not necessarily commute. This approximation is very useful in Hamiltonian simulation since Hamiltonian usaully consists of a sum of non-commuting terms. The error term can be surpressed by increasing the number of times these expotential function are multiplied together, such that:
#### $e^{\delta(A+B)} = (e^{\delta \frac{A}{n}} e^{\delta \frac{B}{n}} + \mathcal{O}((\frac{\delta}{n})^2)^n$
#### The error term can then be made arbritrarily small by simply increasing the number n, although that of course increases the depth of the circuit.

#### In a specific case of the 1D Ising model, where $H = J \sum_{i=0}^m Z_i Z_{i+1}$ and m is the number of sites in the system. For this case the time evolution operator is given by:
#### $e^{-iHt} = e^{-i J \sum_{i=0}^m (Z_i Z_{i+1}) t} \approx (\Pi_i^m e^{-iJZ_i \frac{t}{n}} e^{-iJZ_{i+1} \frac{t}{n}})^n$
#### Here we see how this approximation works by turning the complicated Ising model time evolution operator into a product of individual time evolution operators that can be implemented in series. That is the large m qubit operator can be turned into a product of n 2 qubit operators.

# Initialising packages and functions

In [1]:
import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt
from functools import reduce

In [2]:
def kron_all(*gates):
    """
    Computes the Kronecker product of multiple gates.
    Flattens nested lists of gates automatically.
    
    Example:
        kron_all([[A, B], C]) -> np.kron(np.kron(A, B), C)
    """
    # Flatten the input
    flat = []
    for g in gates:
        if isinstance(g, list):
            flat.extend(g)
        else:
            flat.append(g)
    # Compute Kronecker product
    return reduce(np.kron, flat)

def gate_pow(gate, n):
    """
    Returns a list of 'gate' repeated n times.
    If you want the actual Kronecker product of the gate with itself n times,
    you can use `kron_all([gate]*n)` instead.
    """
    return [gate] * n

In [9]:
class QuantumCircuit:
    def __init__(self, n):
        '''
        Initialises a quantum circuit in the 0 state with n qubits
        '''
        self.n = n
        self.state = np.zeros(2**n, dtype=np.complex128)
        self.state[0] = 1.0

    def __repr__(self):
        '''
        Makes itself printable
        '''
        return f"Statevector=\n{self.state}"

    def one_gate(self, gate, n):
        '''
        Apply any gate to qubit n in circuit
        '''
        I = np.identity(2)                #identity
        q = int(np.log2(len(self.state))) #number of qubits

        #tensor product between identity and gate to broadcast to circuit size
        operator = kron_all(
            gate_pow(I, n),
            gate,
            gate_pow(I, q - n -1) 
        ) 

        self.state = operator @ self.state

    def measure(self, n):
        '''
        Measure qubit n in z basis
        '''
        I = np.identity(2)                #identity
        zero = np.array([1, 0])           #zero state
        one = np.array([0,1])             #one state
        rng = np.random.default_rng()     #setup random number generator
        q = int(np.log2(len(self.state))) #number of qubits

        #projector onto the 0 state 
        projector_0 = kron_all(
            gate_pow(I, n), 
            np.outer(zero, zero), 
            gate_pow(I, q - n - 1)
        )

        #projector onto the 1 state 
        projector_1 = kron_all(
            gate_pow(I, n),
            np.outer(one, one),
            gate_pow(I, q - n - 1)
            )

        state_0 = projector_0 @ self.state
        state_1 = projector_1 @ self.state

        p = [np.linalg.norm(state_0) ** 2, np.linalg.norm(state_1) ** 2]
        
        output = rng.choice((0,1), p = p)
        if output:
            self.state = state_1 / np.sqrt(p[1])
        else:
            self.state = state_0 / np.sqrt(p[0])
        print(f"Qubit {n} is measured to be {output}, congratulations!")
        return output

In [11]:
qc = QuantumCircuit(2)
X = np.array([[0,1],[1,0]])
H = 1/np.sqrt(2) * np.array([[1,1],[1,-1]])

qc.one_gate(X, 0)
qc.one_gate(H, 0)
output = qc.measure(0)
print(qc)

Qubit 0 is measured to be 0, congratulations!
Statevector=
[1.+0.j 0.+0.j 0.+0.j 0.+0.j]


# Simulations and plots

# Discussion and conclussion